In [ ]:
import numpy as np
import os
%matplotlib widget
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import seaborn as sns
import pybaselines
import scipy.ndimage
import scipy.fft
sns.set_theme()

xlen = 252
ylen = 234
xsize = 0.01175 / xlen
ysize = 0.01095 / ylen

floorplanBlocks  = {
"SM0_EXE_T" : [0.0007802371541501976,0.0004473191489361703,0.009808695652173911,0.005288617021276596],
"SM0_EXE_B" : [0.0007802371541501976,0.0004473191489361702,0.009808695652173911,0.004193617021276596],
"SM0_I_CACHE" : [0.0008359683794466403,0.0006476808510638298,0.00787203557312253,0.004640936170212766],
"SM0_MEM_T" : [0.0021038537549407112,0.0004473191489361703,0.0077048418972332005,0.005288617021276596],
"SM0_MEM_B" : [0.0021038537549407112,0.0004473191489361702,0.0077048418972332005,0.004193617021276596],
"SM0_REG_FILE" : [0.0009056324110671936,0.0006476808510638298,0.009683300395256916,0.004640936170212766],
"SM0_SHD_MEM" : [0.000975296442687747,0.0006476808510638298,0.00870800395256917,0.004640936170212766],
"SM1_EXE_T" : [0.0007802371541501976,0.0004473191489361703,0.0005108695652173913,0.0053352127659574465],
"SM1_EXE_B" : [0.0007802371541501976,0.0004473191489361702,0.0005108695652173913,0.004240212765957447],
"SM1_I_CACHE" : [0.0008359683794466403,0.0006476808510638298,0.0023917984189723318,0.004687531914893617],
"SM1_MEM_T" : [0.0021038537549407112,0.0004473191489361703,0.001291106719367589,0.0053352127659574465],
"SM1_MEM_B" : [0.0021038537549407112,0.0004473191489361702,0.001291106719367589,0.004240212765957447],
"SM1_REG_FILE" : [0.0009056324110671936,0.0006476808510638298,0.0005108695652173913,0.004687531914893617],
"SM1_SHD_MEM" : [0.000975296442687747,0.0006476808510638298,0.001416501976284585,0.004687531914893617],
"SM2_EXE_T" : [0.0007802371541501976,0.0004473191489361703,0.009808695652173911,0.007245638297872341],
"SM2_EXE_B" : [0.0007802371541501976,0.0004473191489361702,0.009808695652173911,0.0061506382978723404],
"SM2_I_CACHE" : [0.0008359683794466403,0.0006476808510638298,0.00787203557312253,0.006597957446808511],
"SM2_MEM_T" : [0.0021038537549407112,0.0004473191489361703,0.0077048418972332005,0.007245638297872341],
"SM2_MEM_B" : [0.0021038537549407112,0.0004473191489361702,0.0077048418972332005,0.0061506382978723404],
"SM2_REG_FILE" : [0.0009056324110671936,0.0006476808510638298,0.009683300395256916,0.006597957446808511],
"SM2_SHD_MEM" : [0.000975296442687747,0.0006476808510638298,0.00870800395256917,0.006597957446808511],
"SM3_EXE_T" : [0.0007802371541501976,0.0004473191489361703,0.0005108695652173913,0.007245638297872341],
"SM3_EXE_B" : [0.0007802371541501976,0.0004473191489361702,0.0005108695652173913,0.0061506382978723404],
"SM3_I_CACHE" : [0.0008359683794466403,0.0006476808510638298,0.0023917984189723318,0.006597957446808511],
"SM3_MEM_T" : [0.0021038537549407112,0.0004473191489361703,0.001291106719367589,0.007245638297872341],
"SM3_MEM_B" : [0.0021038537549407112,0.0004473191489361702,0.001291106719367589,0.0061506382978723404],
"SM3_REG_FILE" : [0.0009056324110671936,0.0006476808510638298,0.0005108695652173913,0.006597957446808511],
"SM3_SHD_MEM" : [0.000975296442687747,0.0006476808510638298,0.001416501976284585,0.006597957446808511],
"SM4_EXE_T" : [0.0007802371541501976,0.0004473191489361703,0.009808695652173911,0.009249255319148936],
"SM4_EXE_B" : [0.0007802371541501976,0.0004473191489361702,0.009808695652173911,0.008154255319148936],
"SM4_I_CACHE" : [0.0008359683794466403,0.0006476808510638298,0.00787203557312253,0.008601574468085106],
"SM4_MEM_T" : [0.0021038537549407112,0.0004473191489361703,0.0077048418972332005,0.009249255319148936],
"SM4_MEM_B" : [0.0021038537549407112,0.0004473191489361702,0.0077048418972332005,0.008154255319148936],
"SM4_REG_FILE" : [0.0009056324110671936,0.0006476808510638298,0.009683300395256916,0.008601574468085106],
"SM4_SHD_MEM" : [0.000975296442687747,0.0006476808510638298,0.00870800395256917,0.008601574468085106],
"SM5_EXE_T" : [0.0007802371541501976,0.0004473191489361703,0.0005573122529644269,0.009202659574468086],
"SM5_EXE_B" : [0.0007802371541501976,0.0004473191489361702,0.0005573122529644269,0.008107659574468086],
"SM5_I_CACHE" : [0.0008359683794466403,0.0006476808510638298,0.0024382411067193676,0.008554978723404256],
"SM5_MEM_T" : [0.0021038537549407112,0.0004473191489361703,0.0013375494071146245,0.009202659574468086],
"SM5_MEM_B" : [0.0021038537549407112,0.0004473191489361702,0.0013375494071146245,0.008107659574468086],
"SM5_REG_FILE" : [0.0009056324110671936,0.0006476808510638298,0.0005573122529644269,0.008554978723404256],
"SM5_SHD_MEM" : [0.000975296442687747,0.0006476808510638298,0.0014629446640316206,0.008554978723404256],
"L2" : [0.00215,0.00540976595744680930,0.0045,0.004240212765957447],
"MC0" : [0.005898221343873518,0.0006523404255319149,0.005712450592885375,4.6595744680851065e-05],
"MC1" : [0.000650197628458498,0.010111276595744682,0.011053359683794466,0.000745531914893617],
"MC2" : [0.010867588932806324,0.000745531914893617,4.644268774703557e-05,0.010157872340425532] }

hwDir = 'hw_heatmaps'
simDir = 'no_secondary'
Rodinia = ['backprop','bfs','b+tree','dwt2d', 'gaussian',
           'heartwall','hotspot', 'kmeans','lavaMD',
           'lud','myocyte','nw']

hwAvg = np.zeros((234,252))
simAvg = np.zeros((234,252))

#load in heatmap
simHeatmaps = {}
hwHeatmaps  = {}
for bm in Rodinia:
    # simulated heatmap
    filename = os.path.join(simDir,(bm + '.grid.steady'))
    fp = open(filename, "r")
    temp = []
    found = False
    for line in fp:
        if line == "\n":
            continue
        temp.append(float(line.strip().split()[1]))
    '''
    for line in fp:
        if found:
            if line.strip().split()[0] == "Layer":
                found = False
                break
            temp.append(float(line.strip().split()[1]))
        else:
            if line.strip() == "Layer 5:":
                found = True
    '''
    temp = np.reshape(temp,(256,256))
    temp = np.array(Image.fromarray(temp).resize((252,234)))
    temp -= 273.15
    simHeatmaps[bm] = temp
    simAvg += temp

    #read hw heatmap
    filename = os.path.join(hwDir,(bm + '_heatmap.csv'))
    temp = pd.read_csv(filename,header=None).to_numpy()
    hwHeatmaps[bm] = temp
    hwAvg += temp

temp
simAvg /= len(Rodinia)
hwAvg /= len(Rodinia)

#hwtemp = (hwAvg-hwAvg.min().min()) / (hwAvg.max().max()-hwAvg.min().min())
#simAvg = (simAvg-simAvg.min().min()) / (simAvg.max().max()-simAvg.min().min())

In [ ]:
fig,ax = plt.subplots(2,len(Rodinia))
percentErrors = {}
for bm,index in zip(Rodinia,range(0,len(Rodinia))):
    
    #Row 1: Heatmap of real gpu
    hw_base = hwHeatmaps[bm]
    sim = simHeatmaps[bm]
    workloadMin = min(hw_base.min().min(),sim.min().min())
    workloadMax = max(hw_base.max().max(),sim.max().max())

    hwtemp = (hw_base-hw_base.min().min()) / (hw_base.max().max()-hw_base.min().min())

    #baseline = pybaselines.optimizers.collab_pls(hwtemp.T,method='iarpls',lam=10e5)[0].T
    #baseline = scipy.ndimage.gaussian_filter(baseline,10)
    #hwtemp = (hwtemp - baseline) + baseline.mean().mean()
    im = ax[0][index].imshow(hwtemp, interpolation='nearest',cmap=plt.get_cmap('inferno'))
    ax[0][index].set_title(bm,fontsize=14)
    ax[0][index].set_yticks([ ])
    ax[0][index].set_xticks([ ])
    if index % len(Rodinia) ==0:
        ax[0][index].set_ylabel('Baseline',fontsize=16)
    im.set_clim(0,1)
    '''
    #Row 2: Difference Map of real GPU
    #comparing differences between benchmarks hardware thermals 
    hw_diff = hw_base.copy()
    hw_diff -= (hwAvg)
    avg = (hw_diff.min().min() + hw_diff.max().max()) / 2
    ran = (hw_diff.max().max() - hw_diff.min().min()) / 2
    hw_diff = (hw_diff - avg) / ran
    ax[1][index].set_yticks([ ])
    ax[1][index].set_xticks([ ])
    if index % len(Rodinia) ==0:
        ax[1][index].set_ylabel('HW Diff',fontsize=14)
    im = ax[1][index].imshow(hw_diff, interpolation='nearest',cmap=plt.get_cmap('seismic'))
    im.set_clim(-1,1)
    '''
    #Row 3: heatmap of gpgpusim
    #sim = pd.read_csv(steadyDir + bm + '_' + flp + '.grid.steady',header=None)
    ax[1][index].set_yticks([ ])
    ax[1][index].set_xticks([ ])
    if index % len(Rodinia) ==0:
        ax[1][index].set_ylabel('GPUCalorie',fontsize=14)
    simtemp = (sim-sim.min().min()) / (sim.max().max()-sim.min().min())
 
    im = ax[1][index].imshow(simtemp, interpolation='nearest',cmap=plt.get_cmap('inferno')) 
    im.set_clim(0,1)

    ''' 
    #Row 4: Difference map ofgpgpusim
    sim_diff = sim.copy()
    sim_diff -= (simAvg)
    avg = (sim_diff.min().min() + sim_diff.max().max()) / 2
    ran = (sim_diff.max().max() - sim_diff.min().min()) / 2
    sim_diff = (sim_diff - avg) / ran
    ax[3][index].set_yticks([ ])
    ax[3][index].set_xticks([ ])
    if index % len(Rodinia) ==0:
        ax[3][index].set_ylabel('GPUCalorie Diff',fontsize=14)
    im = ax[3][index].imshow(sim_diff, interpolation='nearest',cmap=plt.get_cmap('seismic'))
    im.set_clim(-1,1)
    '''
    #diff = np.linalg.norm((hw_diff - sim_diff).to_numpy(),ord=2) / np.linalg.norm(hw_diff.to_numpy(),ord=2)
    diff = np.linalg.norm((hw_base - sim),ord=2) / np.linalg.norm(hw_base,ord=2)
    #diff = abs(hw_base.mean().mean() - sim.mean().mean()) / ((hw_base.mean().mean() + sim.mean().mean())/ 2)

    print(bm+',' + str(hw_base.min().min())  +','+
                    str(hw_base.mean().mean())  +','+
                    str(hw_base.max().max())  +','+
                    str(sim.min().min())  +','+
                   str(sim.mean().mean())  +','+
                    str(sim.max().max())  +','+
                    str(diff*100))

    #end for flpt
#end for bm
#fig.savefig("floorplan_results.pdf",bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,len(Rodinia),sharey=False)
for bm,index in zip(Rodinia,range(0,len(Rodinia))):
    #Heatmap of real gpu
    hw = hwHeatmaps[bm]
    #heatmap of gpgpusim
    sim = simHeatmaps[bm]

    hw_avg = []
    sim_avg = []
    sm = []
    unit = []
    for key in floorplanBlocks:
        x = int(floorplanBlocks[key][2]//xsize)
        width = int((floorplanBlocks[key][0]//xsize))
        y = int(floorplanBlocks[key][3]//ysize)
        length = int((floorplanBlocks[key][1]//ysize))

        hw_block = hw[y:y+length,x:x+width]
        sim_block = sim[y:y+length,x:x+width] 

        hw_avg.append(hw_block.mean().mean())
        sim_avg.append(sim_block.mean().mean())
        temp = key.split("_")
        sm.append(temp[0])
        if len(temp) > 1:
            unit.append(key.split("_")[1])
        else:
            unit.append("MEM")


    data = pd.DataFrame(
        dict(
            Baseline=hw_avg,
            GPUCalorie=sim_avg,
            sm=sm,
            unit=unit
        )
    )
    sns.scatterplot(data=data, x="Baseline", y="GPUCalorie",hue="sm",style="unit",ax=ax[index],legend=False)
    lims = [
    np.min([ax[index].get_xlim(), ax[index].get_ylim()]),  # min of both axes
    np.max([ax[index].get_xlim(), ax[index].get_ylim()]),  # max of both axes
    ]
    ax[index].set_xlim(lims)
    ax[index].set_ylim(lims)
    ax[index].set_ylabel('')
    ax[index].set_xlabel('')
   # ax[index].set_xticks([])
   # ax[index].set_yticks([])
    ax[index].set_title(bm)


    ax[index].plot(lims, lims, 'b--', alpha=0.25,zorder=0)
    ax[index].set_aspect('equal')
plt.show()

In [ ]:

fig, ax = plt.subplots(1,sharey=False)
error = []
sm = []
unit = []
benchmark = []
for bm,index in zip(Rodinia,range(0,len(Rodinia))):
    #Heatmap of real gpu
    hw = hwHeatmaps[bm]
    #heatmap of gpgpusim
    sim = simHeatmaps[bm]

    for key in floorplanBlocks:
        x = int(floorplanBlocks[key][2]//xsize)
        width = int((floorplanBlocks[key][0]//xsize))
        y = int(floorplanBlocks[key][3]//ysize)
        length = int((floorplanBlocks[key][1]//ysize))

        hw_block = hw[y:y+length,x:x+width]
        sim_block = sim[y:y+length,x:x+width] 

        hwMean = (hw_block.mean().mean())
        simMean = (sim_block.mean().mean())
        err = abs(simMean - hwMean) / abs(hwMean)
        error.append(err)
        temp = key.split("_")
        sm.append(temp[0])
        if len(temp) > 1:
            unit.append(key.split("_")[1])
        else:
            unit.append("MEM")
        benchmark.append(bm)


data = pd.DataFrame(
    dict(
        Error=error,
        Benchmarks=benchmark,
        sm=sm,
        unit=unit
    )
)
sns.stripplot(data=data, x="Benchmarks", y="Error",hue="sm",size=4.5,ax=ax,alpha=.75)
#ax[1].set_ylim([0.0 , 0.3])
#ax[1].get_legend().remove()
ax.set_xlabel("")


#sns.stripplot(data=data, x="Benchmarks", y="Error",hue="sm",size=4.5,ax=ax[0],alpha=.75)
#ax[0].set_ylim([0.5 , 0.8])
#ax[0].set_xticks([])
#ax[0].set_xlabel("")
#ax[0].set_ylabel("")
#ax[0].get_legend().remove()


#d = .5  # proportion of vertical to horizontal extent of the slanted line
#kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
#              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
#ax[0].plot([0, 1], [0, 0], transform=ax[0].transAxes, **kwargs)
#ax[1].plot([0, 1], [1, 1], transform=ax[1].transAxes, **kwargs)


plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,sharey=False)
temperatures = []
benchmarks = []
platforms = []
for bm,index in zip(Rodinia,range(0,len(Rodinia))):
    #Heatmap of real gpu
    temp = list(hwHeatmaps[bm].flatten())
    temperatures  += temp
    benchmarks += [bm for x in range(len(temp))]
    platforms += ["Baseline" for x in range(len(temp))]
    
    #heatmap of gpgpusim
    temp = list(simHeatmaps[bm].flatten())
    temperatures  += temp
    benchmarks += [bm for x in range(len(temp))]
    platforms += ["GPUCalorie" for x in range(len(temp))]


data = pd.DataFrame(
    dict(
        Temperature=temperatures,
        Benchmarks=benchmarks,
        Legend=platforms
    )
)


sns.violinplot(data=data, x="Benchmarks", y="Temperature", hue="Legend", split=True)
ax.set_xlabel("")
plt.show()
